In [7]:
import pandas as pd

# Load data
meta = pd.read_excel('/content/meta.xlsx')
reviews = pd.read_excel('/content/reviews.xlsx')


ModuleNotFoundError: No module named 'pandas'

In [ ]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

def jaccard_similarity(text1, text2):
    stop_words = set(stopwords.words('english'))

    set1 = set([word for word in str(text1).lower().split() if word not in stop_words])
    set2 = set([word for word in str(text2).lower().split() if word not in stop_words])

    if not set1 or not set2:
        return 0.0  # Avoid division by zero

    intersection = set1.intersection(set2)
    union = set1.union(set2)

    return len(intersection) / len(union)

results = []

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:


for index, row in reviews.iterrows():
    subfolder = row['subfolder']
    rid = row['rid']
    existing_confidence = row['confidence']

    try:
        paper_match = meta[meta['Folder Name'] == subfolder]
        if not paper_match.empty:
            paper_abstract = paper_match['abstract'].iloc[0]
            paper_title = paper_match['title'].iloc[0]
            text1 = str(paper_abstract) + ' ' + str(paper_title)
            text2 = str(row['paper_summary']) + ' ' + str(row['summary_of_strengths']) + ' ' + str(row['summary_of_weaknesses'])

            # Compute Jaccard similarity
            jaccard_score = jaccard_similarity(text1, text2)

            # Scale to 1–5 range
            scaled_score = round(1 + (jaccard_score * 4), 2)

            results.append({
                'subfolder': subfolder,
                'rid': rid,
                'existing_confidence': existing_confidence,
                'jaccard_similarity': jaccard_score,
                'scaled_similarity_1_to_5': scaled_score
            })
        else:
            print(f"No matching paper found for subfolder: {subfolder}")
            results.append({
                'subfolder': subfolder,
                'rid': rid,
                'existing_confidence': existing_confidence,
                'jaccard_similarity': None,
                'scaled_similarity_1_to_5': None
            })

    except Exception as e:
        print(f"Error processing row {index}: {e}")
        results.append({
            'subfolder': subfolder,
            'rid': rid,
            'existing_confidence': existing_confidence,
            'jaccard_similarity': None,
            'scaled_similarity_1_to_5': None
        })

results_df = pd.DataFrame(results)
results_df


,subfolder,rid,existing_confidence,jaccard_similarity,scaled_similarity_1_to_5
0,37baabcc12717c1fefe6779955e7680c615a878a69d45d...,5c4701c5eabb6f35df25974c92a58e5ae949617a98de91...,4,0.146018,1.58
1,40457e3287163a0a879367375dbd3ed58537a0de494bf3...,a62942bb748a19299a4b0ca74615c6ffb02b6fdaae9fc4...,4,0.130631,1.52
2,ececaa7e6b60520b8b22416cd0ad34ab381fea40c776d7...,ded521e21be436886d381b7dfa702e2266ce7a24d5b57c...,3,0.245989,1.98
3,4be53da06598da6910f11d0c86d5f2180616d1ebdab941...,da580da9a5cedb9f9d5fd02da42ff42e9c95aac36bc7de...,5,0.181818,1.73
4,4be53da06598da6910f11d0c86d5f2180616d1ebdab941...,df84af6bd18973895025734688345e5d694e2b90bc8517...,4,0.099502,1.40
...,...,...,...,...,...
1420,bc6e70bea0ae4b6a0cdc89bf2e2123138e6ed4208ea52d...,fc3cfc095d65f0ce51cd7689181efb7608be4a6806ed82...,4,0.104895,1.42
1421,3e5a5d13187e62125dabfa37eaccff24d432044f8c13bc...,ad727c2a4fef8c8c5efb5b3a4b2998394c14e6abda5e21...,4,0.097421,1.39
1422,12f874be803608fee90c23bffa956408e9b5830f25ed96...,6a6aaa2db774688d9550b190e30e1aa3ce6e02decd5e10...,3,0.132450,1.53
1423,07f5ffd24aaa1fc1d5a86219c2f34f3e5e276d3894dfde...,5d6c46bc13df3960d2eb20699135c2582fa5143169d633...,2,0.080460,1.32


In [ ]:
# Calculate the correlation
correlation = results_df['existing_confidence'].corr(results_df['scaled_similarity_1_to_5'], method='pearson')

print(f"Correlation between existing_confidence and scaled_similarity_1_to_5: {correlation}")


Correlation between existing_confidence and scaled_similarity_1_to_5: -0.09898805693001077


In [ ]:

import pandas as pd

# Load the Excel file
reviews_df = pd.read_excel('/content/reviews_with_bert.xlsx')

# Add the 'jaccard_confidence' column with the values from 'scaled_similarity_1_to_5'
reviews_df['jaccard_confidence'] = results_df['scaled_similarity_1_to_5']

# Save the updated DataFrame to a new Excel file
reviews_df.to_excel('/content/reviews_with_bert_jaccard.xlsx', index=False)
